In [1]:
import requests

In [2]:
url='https://data.epa.gov.tw/api/v2/aqx_p_02?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=datacreationdate%20desc&format=JSON'

In [3]:
resp=requests.get(url)
resp


<Response [200]>

### 取得資料所在位置

In [4]:
datas=resp.json()['records']
datas


[{'site': '大城',
  'county': '彰化縣',
  'pm25': '11',
  'datacreationdate': '2023-05-04 12:00',
  'itemunit': 'μg/m3'},
 {'site': '富貴角',
  'county': '新北市',
  'pm25': '18',
  'datacreationdate': '2023-05-04 12:00',
  'itemunit': 'μg/m3'},
 {'site': '麥寮',
  'county': '雲林縣',
  'pm25': '8',
  'datacreationdate': '2023-05-04 12:00',
  'itemunit': 'μg/m3'},
 {'site': '關山',
  'county': '臺東縣',
  'pm25': '3',
  'datacreationdate': '2023-05-04 12:00',
  'itemunit': 'μg/m3'},
 {'site': '馬公',
  'county': '澎湖縣',
  'pm25': '9',
  'datacreationdate': '2023-05-04 12:00',
  'itemunit': 'μg/m3'},
 {'site': '金門',
  'county': '金門縣',
  'pm25': '8',
  'datacreationdate': '2023-05-04 12:00',
  'itemunit': 'μg/m3'},
 {'site': '馬祖',
  'county': '連江縣',
  'pm25': '14',
  'datacreationdate': '2023-05-04 12:00',
  'itemunit': 'μg/m3'},
 {'site': '埔里',
  'county': '南投縣',
  'pm25': '17',
  'datacreationdate': '2023-05-04 12:00',
  'itemunit': 'μg/m3'},
 {'site': '復興',
  'county': '高雄市',
  'pm25': '14',
  'datacreationd

In [5]:
len(datas)

78

In [6]:
datas[0]

{'site': '大城',
 'county': '彰化縣',
 'pm25': '11',
 'datacreationdate': '2023-05-04 12:00',
 'itemunit': 'μg/m3'}

### 建立資料庫  
- 1.創建資料庫
- 2.建立資料表
- 3.寫入不重複資料
- 4.取最新數據

In [7]:
import sqlite3

In [8]:
conn=sqlite3.connect('pm25.db')
cursor=conn.cursor()

conn,cursor

(<sqlite3.Connection at 0x23288b5ae40>, <sqlite3.Cursor at 0x23288bad420>)

In [9]:
datas[0].keys()

dict_keys(['site', 'county', 'pm25', 'datacreationdate', 'itemunit'])

In [10]:
sqlstr='''
create table if not exists data(
id integer primary key autoincrement,
site text,
county text,
pm25 integer,
datacreationdate datetime,
itemunit text
)

'''
cursor.execute(sqlstr)
conn.commit()

### 檢查資料是否存在

In [11]:
# 組合內容
values = []
for data in datas:
    data = list(data.values())
    try:
        data[2] = eval(data[2])
        values.append(data)
    except Exception as e:
        print(e)
values

unexpected EOF while parsing (<string>, line 0)
unexpected EOF while parsing (<string>, line 0)
unexpected EOF while parsing (<string>, line 0)


[['大城', '彰化縣', 11, '2023-05-04 12:00', 'μg/m3'],
 ['富貴角', '新北市', 18, '2023-05-04 12:00', 'μg/m3'],
 ['麥寮', '雲林縣', 8, '2023-05-04 12:00', 'μg/m3'],
 ['關山', '臺東縣', 3, '2023-05-04 12:00', 'μg/m3'],
 ['馬公', '澎湖縣', 9, '2023-05-04 12:00', 'μg/m3'],
 ['金門', '金門縣', 8, '2023-05-04 12:00', 'μg/m3'],
 ['馬祖', '連江縣', 14, '2023-05-04 12:00', 'μg/m3'],
 ['埔里', '南投縣', 17, '2023-05-04 12:00', 'μg/m3'],
 ['復興', '高雄市', 14, '2023-05-04 12:00', 'μg/m3'],
 ['永和', '新北市', 41, '2023-05-04 12:00', 'μg/m3'],
 ['竹山', '南投縣', 32, '2023-05-04 12:00', 'μg/m3'],
 ['中壢', '桃園市', 19, '2023-05-04 12:00', 'μg/m3'],
 ['三重', '新北市', 26, '2023-05-04 12:00', 'μg/m3'],
 ['冬山', '宜蘭縣', 6, '2023-05-04 12:00', 'μg/m3'],
 ['宜蘭', '宜蘭縣', 9, '2023-05-04 12:00', 'μg/m3'],
 ['陽明', '臺北市', 12, '2023-05-04 12:00', 'μg/m3'],
 ['花蓮', '花蓮縣', 7, '2023-05-04 12:00', 'μg/m3'],
 ['臺東', '臺東縣', 4, '2023-05-04 12:00', 'μg/m3'],
 ['恆春', '屏東縣', 4, '2023-05-04 12:00', 'μg/m3'],
 ['潮州', '屏東縣', 8, '2023-05-04 12:00', 'μg/m3'],
 ['屏東', '屏東縣', 15, '2023-05-0

In [12]:
sqlstr=f'select * from data where site="{data[0]}" and datacreationdate="{data[-2]}"'
sqlstr

'select * from data where site="基隆" and datacreationdate="2023-05-04 12:00"'

In [13]:
list(cursor.execute(sqlstr))

[(75, '基隆', '基隆市', 29, '2023-05-04 12:00', 'μg/m3')]

In [14]:
sqlstr=f'insert into data(site,county,pm25,datacreationdate,itemunit) values("{data[0]}","{data[1]}",\
    {data[2]},"{data[3]}","{data[4]}")'
sqlstr

'insert into data(site,county,pm25,datacreationdate,itemunit) values("基隆","基隆市",    29,"2023-05-04 12:00","μg/m3")'

### 寫入資料

In [15]:
# 組合內容
count=0
for data in datas:
    data = list(data.values())
    try:
        data[2] = eval(data[2])
        sqlstr=f'select * from data where site="{data[0]}" and datacreationdate="{data[-2]}"'
        result=list(cursor.execute(sqlstr))    
        if result==[]:
            sqlstr=f'insert into data(site,county,pm25,datacreationdate,itemunit) values("{data[0]}","{data[1]}",\
            {data[2]},"{data[3]}","{data[4]}")'
            cursor.execute(sqlstr)
            count+=1
    except Exception as e:
        pass
        
conn.commit()
print(f'更新:{count}筆資料.')

更新:0筆資料.


### 六都平均值

In [16]:
import pandas as pd

In [17]:
df=pd.DataFrame(datas)
df

,site,county,pm25,datacreationdate,itemunit
0,大城,彰化縣,11,2023-05-04 12:00,μg/m3
1,富貴角,新北市,18,2023-05-04 12:00,μg/m3
2,麥寮,雲林縣,8,2023-05-04 12:00,μg/m3
3,關山,臺東縣,3,2023-05-04 12:00,μg/m3
4,馬公,澎湖縣,9,2023-05-04 12:00,μg/m3
...,...,...,...,...,...
73,土城,新北市,32,2023-05-04 12:00,μg/m3
74,新店,新北市,36,2023-05-04 12:00,μg/m3
75,萬里,新北市,15,2023-05-04 12:00,μg/m3
76,汐止,新北市,,2023-05-04 12:00,μg/m3


In [18]:
set(df['county'])

{'南投縣',
 '嘉義市',
 '嘉義縣',
 '基隆市',
 '宜蘭縣',
 '屏東縣',
 '彰化縣',
 '新北市',
 '新竹市',
 '新竹縣',
 '桃園市',
 '澎湖縣',
 '臺中市',
 '臺北市',
 '臺南市',
 '臺東縣',
 '花蓮縣',
 '苗栗縣',
 '連江縣',
 '金門縣',
 '雲林縣',
 '高雄市'}

In [19]:
six_countys=['臺北市','新北市','桃園市','臺中市','臺南市','高雄市']

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78 entries, 0 to 77
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   site              78 non-null     object
 1   county            78 non-null     object
 2   pm25              78 non-null     object
 3   datacreationdate  78 non-null     object
 4   itemunit          78 non-null     object
dtypes: object(5)
memory usage: 3.2+ KB


In [21]:
df.groupby('county').get_group(six_countys[2])['pm25'].astype(int).mean()

ValueError: invalid literal for int() with base 10: ''

In [22]:
six_pm25={}

for county in six_countys:
    six_pm25[county]=round(df.groupby('county').get_group(county)['pm25'].astype(int).mean(),1)
    
six_pm25

ValueError: invalid literal for int() with base 10: ''

### 每個縣市站點資訊

In [23]:
print(list(set(df['county'])))

['臺中市', '臺東縣', '高雄市', '彰化縣', '金門縣', '新竹縣', '新北市', '嘉義市', '新竹市', '基隆市', '苗栗縣', '臺北市', '連江縣', '宜蘭縣', '桃園市', '澎湖縣', '嘉義縣', '雲林縣', '臺南市', '屏東縣', '南投縣', '花蓮縣']


In [24]:
df.groupby('county').get_group('新北市')[['site','pm25']].values.tolist()

[['富貴角', '18'],
 ['永和', '41'],
 ['三重', '26'],
 ['淡水', '12'],
 ['林口', '13'],
 ['菜寮', '28'],
 ['新莊', '23'],
 ['板橋', '39'],
 ['土城', '32'],
 ['新店', '36'],
 ['萬里', '15'],
 ['汐止', '']]